**DTI DATA  / Format and save the 2 files DTI and Subjet after data management and process**

In [114]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from itertools import combinations
from math import log
import scipy.stats as scs
from statsmodels.sandbox.stats.multicomp import multipletests
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
import warnings
import array as arr
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d  

dti_metric_list_str=['FA','RD','AD','MD']
dti_metric_list=['FA','AD','RD','MD']
CCx_list=['CC_1','CC_2','CC_3','CC_4','CC_5','CC_6','CC_7']
dti_metric=0  #0 : FA / 1 : RD ...

ccxID=5

warnings.filterwarnings('ignore')

Aging_INDICES = pd.read_csv('Aging_INDICES NC.csv', sep=",")
# sub-02_FA_tractometry.csv

SubjectTable=Aging_INDICES[['sex', 'age', 'group', 'JND_P', 'IHD', 'FA_body', 'MD_body','AD_body', 'RD_body']]
SubjectTable.head()



,sex,age,group,JND_P,IHD,FA_body,MD_body,AD_body,RD_body
0,F,25,Y,9.56,0.534027,0.583124,0.000678,0.001200,0.000417
1,M,27,Y,6.69,0.570600,0.613808,0.000638,0.001163,0.000375
2,F,28,Y,12.92,-0.140120,0.627866,0.000631,0.001173,0.000361
3,F,21,Y,1.95,0.488581,0.596784,0.000665,0.001187,0.000404
4,F,25,Y,2.54,0.578001,0.637038,0.000636,0.001197,0.000356


In [116]:
#One hot encoder for groups

category_column = SubjectTable[['group']]
encoder = OneHotEncoder(sparse_output=False)
# Ajuster et transformer la colonne catégorielle
encoded_categories = encoder.fit_transform(category_column)
# Créer un DataFrame avec les colonnes encodées
encoded_df = pd.DataFrame(encoded_categories, columns=encoder.get_feature_names_out(['group']))
# Concaténer le DataFrame encodé avec le DataFrame original (sans la colonne catégorielle)
SubjectTable = pd.concat([SubjectTable.drop(columns=['group','sex']), encoded_df], axis=1)
# Afficher le DataFrame résultant
SubjectTable.head()



imputer = KNNImputer(n_neighbors=2)

SubjectTable_imputed=pd.DataFrame(imputer.fit_transform(SubjectTable))### sans Y et O des groupes initiaux 
SubjectTable_imputed_df=SubjectTable_imputed

# SubjectTable_imputed_df.columns=SubjectTable.columns
SubjectTable_imputed.columns=SubjectTable.columns

SubjectTable_imputed["group"]=Aging_INDICES["group"]
SubjectTable_imputed["sex"]=Aging_INDICES["sex"]

SubjectTable_imputed.to_csv('Subject_preproc.csv', index=False)  
SubjectTable_imputed.head()


,age,JND_P,IHD,FA_body,MD_body,AD_body,RD_body,group_O,group_Y,group,sex
0,25.0,9.56,0.534027,0.583124,0.000678,0.001200,0.000417,0.0,1.0,Y,F
1,27.0,6.69,0.570600,0.613808,0.000638,0.001163,0.000375,0.0,1.0,Y,M
2,28.0,12.92,-0.140120,0.627866,0.000631,0.001173,0.000361,0.0,1.0,Y,F
3,21.0,1.95,0.488581,0.596784,0.000665,0.001187,0.000404,0.0,1.0,Y,F
4,25.0,2.54,0.578001,0.637038,0.000636,0.001197,0.000356,0.0,1.0,Y,F


In [138]:

# %% CREATE CSV allsub DTI FILE FOR GIVEN CC and metric 

dti_metric_list=['FA']
CCx_list=['CC_1']
CCx_list=['CC_1','CC_2','CC_3','CC_4','CC_5','CC_6','CC_7']

tracto_folder_path = './Tractometry_allSubs_New2024/'

listsub=['01','02','03','05','06','08','09','10','11','12','13','14','15','16','17'
         ,'20','21','22','23','24','25','26','27','29','30','31','32','33','34','35','36'
         ,'37','38','39','41','42','43','44','45']

for dti_metric in dti_metric_list:
    for CCx in CCx_list:
        list_CCx_val = []
        for subj in listsub:
            file_path = os.path.join(tracto_folder_path, f'sub-{subj}_{dti_metric}_tractometry.csv')
            data_sub = pd.read_csv(file_path, sep=";")
            list_CCx_val.append(data_sub[CCx].values)
       
        dfCCx = pd.DataFrame(list_CCx_val)
        dfCCx.to_csv(CCx +'_'+dti_metric+'.csv', index=False)  
dfCCx.head()

,0,1,2,3,4,5,6,7,8,9,...,88,89,90,91,92,93,94,95,96,97
0,0.356210,0.377167,0.398177,0.409793,0.419128,0.414688,0.418795,0.426900,0.436085,0.446459,...,0.414558,0.423013,0.407723,0.385159,0.381491,0.370314,0.352353,0.333934,0.307523,0.312345
1,0.417318,0.429404,0.433014,0.432301,0.435782,0.439983,0.453379,0.453749,0.451598,0.449239,...,0.421655,0.401132,0.427388,0.445278,0.434595,0.420337,0.395027,0.398180,0.417838,0.437414
2,0.442530,0.465933,0.476546,0.491333,0.489132,0.476993,0.472322,0.464494,0.457704,0.460150,...,0.486801,0.485924,0.479508,0.490794,0.519386,0.523226,0.538164,0.539816,0.540034,0.565643
3,0.388942,0.391607,0.401572,0.429047,0.472271,0.516702,0.531579,0.520122,0.521654,0.529428,...,0.374642,0.383784,0.415757,0.449650,0.440048,0.400172,0.346409,0.318612,0.309016,0.319818
4,0.425780,0.432215,0.441219,0.448596,0.453740,0.452137,0.451609,0.454536,0.440034,0.426992,...,0.553321,0.547069,0.529026,0.518297,0.501368,0.487575,0.476482,0.477705,0.470368,0.463595
